- Autor: Virginia Monroy Malca
- Correo: viriginia.monroy@ibm.com

# Usando Machine Learning 

## Importando Token del proyecto

## Importando el dataset

In [2]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [17]:
%%time
#2
## Importar datos aquí

In [ ]:
#3
data.shape

In [ ]:
#4
feature_col=list(set(data.columns.tolist()) - set(['Churn_Label']))
##Define X and Y
X=data[feature_col]
y=data.Churn_Label

## Preprocesamiento de los datos

### Missings

In [ ]:
#5
## Validando que ya no hayan datos nulos
import numpy as np
np.sum(X.isnull())

### Dummies

In [ ]:
%%time
#7
X = pd.get_dummies(X)

### PCA

In [ ]:
%%time
#8
# Descargando librería
from sklearn.decomposition import PCA
# Generando objeto
pca=PCA(n_components=33)
# Entrenando y aplicando al dataset
data_pca=pca.fit_transform(X.iloc[:,3:])
df_pca = pd.DataFrame(data_pca)
data_final = pd.concat([X, df_pca], axis=1)
X = data_final
X.head()

## Data para ML

In [ ]:
#9
y.replace(to_replace=['No', 'Yes'], value=[0, 1], inplace = True)
y.head()

## Xgboost (correr con cpu)

### Importando las liberías

In [14]:
#10
##Import libraries
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
#Librerás para las métricas
from sklearn.metrics import accuracy_score, f1_score, recall_score, auc, precision_score, roc_auc_score
#Librería para obtener la fecha actual
from datetime import date

### Muestreo Estratificado

In [ ]:
#11
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=123)
print ("DONE")

### Construyendo el XGBoost

In [ ]:
%%time
#12
#Crear una matriz admitida para el xgboost (solo acepta este formato)
dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test,label=y_test)
# revisamos el target y si las categorías están balanceadas
data['Churn_Label'].value_counts()
params={'objetive':'binary_logistic','max_depth':'3','learning_rate': '0.01', 'seed':123,
        'num_leaves':'127','min_data_leaf':2660,'grow_policy':'lossguide'}
params['eval_metric']='auc'
###AÑADIENDO PARAMETRO DE PESO
#igualar el 1 a cero : peso artificial
params['scale_pos_weight']=5174/1869

##lista de evaluacion -> durante cada iteracion del modelo: para que evalue el auc del modelo. 
evallist=[(dtest,'validacion'),(dtrain,'train')]

#Para determinar el num de arboles que se va utilizar la evaluacion en el test (AUC por ejemplo) 
num_round=5

##Escribe el auc de la muestra de validacion y la muestra de train
xgBoost=xgb.train(params,dtrain,num_round,evallist)

## Guardando el modelo en WML

In [1]:
#17
from watson_machine_learning_client import WatsonMachineLearningAPIClient
# Instanciar las credenciales
wml_credentials = {"Insertar credenciales aquí"}


client = WatsonMachineLearningAPIClient(wml_credentials)

### Guardar el modelo en un repositorio WML

In [ ]:
#18
published_model = client.repository.store_model(xgBoost, "rg-cp4d-learn-VMM-model")

In [ ]:
#19
client.repository.list_models()